#### This file is used to check id the Froude Number is below 1, subcritical flow

In [1]:
import pandas as pd
import re
import os
import glob

#### Loading Data

In [2]:
######## Check the data path ######## 
examples_path = glob.glob('/Users/ragini/Desktop/Thesis/FullSWOF_2D-1.09.01_win_2/Examples/Model_6_extended_flow/train/*')
# hydrographs = ['2', '4', '6', '8', '10']
evol_paths = [os.path.join(i, 'Outputs/huz_evolution.dat')
             for i in examples_path]

#### Dataframe Parser

In [3]:
def parse_evolution_file_to_datafram(evol_path):
    output =[]
    fp = open(evol_path)
    
    for idx,line in enumerate(fp):
        # Skipping first 5 lines
        if idx<5:
            continue
        # Skipping new lines
        if line =='\n':
            continue
        # From # time extracting time
        elif line[:6] == '# time':
            extract_time = re.findall(r"[-+]?\d*\.\d+|\d+", line)
            
        # Extracting all the input variables and appending time as first element
        else:
            extract_input = extract_time + re.split(r'\t\s*', line.rstrip('\n'))
            output.append(extract_input)

    fp.close()
    
    # From extracted input creating dataframe
    evol_df = pd.DataFrame(output, columns=['time','i','(j-0.5)*dy','h','u','v[i][j]',
                              'h[i][j]+z[i][j]','z[i][j]','norm_U[i][j]','Fr[i][j] (Froude)','qx[i][j]',
                              'qy[i][j]','q'])
    # Dropping columns which are not required
    evol_df = evol_df.drop(columns=['(j-0.5)*dy','v[i][j]',
                              'h[i][j]+z[i][j]','z[i][j]','norm_U[i][j]','qx[i][j]',
                              'qy[i][j]'])

    
    # Converting time to float and sorting based on time
    evol_df['time'] = evol_df['time'].astype(float)
    evol_df['Fr[i][j] (Froude)']= evol_df['Fr[i][j] (Froude)'].astype(float)
    evol_df = evol_df.sort_values('time')
    
    
    # Dropping first row as it doesn't have du_dt
    evol_df = evol_df.drop(0).reset_index(drop=True)
    return evol_df

#### Check for Froude Number

In [4]:
for path in evol_paths:
    tmp = parse_evolution_file_to_datafram(path)
    Froude_num = tmp['Fr[i][j] (Froude)'].max()
    print(Froude_num)

0.86754
0.729636
0.644444
